In [42]:
import pandas as pd
import os

full_data = pd.DataFrame()

for file in os.listdir('./Sales_Data/'):
    df = pd.read_csv('./Sales_Data/'+file)
    full_data = pd.concat([full_data,df])


In [43]:
full_data.isna().mean() * 100

Order ID            0.291678
Product             0.291678
Quantity Ordered    0.291678
Price Each          0.291678
Order Date          0.291678
Purchase Address    0.291678
dtype: float64

In [82]:
from os import replace
from re import split


full_data = full_data.dropna()

full_data = full_data.drop(full_data[full_data['Order Date'] == 'Order Date'].index)

full_data['Order Date'] = pd.to_datetime(full_data['Order Date'])
full_data['Quantity Ordered'] = pd.to_numeric(full_data['Quantity Ordered'])
full_data['Price Each'] = pd.to_numeric(full_data['Price Each'])

full_data['Order Day'] = full_data['Order Date'].dt.day
full_data['Order Month'] = full_data['Order Date'].dt.month_name()
full_data['Order Year'] = full_data['Order Date'].dt.year

full_data['Total Value'] = full_data['Quantity Ordered'] * full_data['Price Each']

full_data['City'] =  full_data['Purchase Address'].str.split(',').str[1].str.replace('City','')

full_data.head()



,Order ID,Product,Quantity Ordered,Price Each,Order Date,Purchase Address,Order Day,Order Month,Order Year,Total Value,City
1,295666,LG Washing Machine,1,600.00,2019-12-29 07:03:00,"562 2nd St, New York City, NY 10001",29,December,2019,600.00,New York
2,295667,USB-C Charging Cable,1,11.95,2019-12-12 18:21:00,"277 Main St, New York City, NY 10001",12,December,2019,11.95,New York
2,176559,Bose SoundSport Headphones,1,99.99,2019-04-07 22:30:00,"682 Chestnut St, Boston, MA 02215",7,April,2019,99.99,Boston
1,150503,AA Batteries (4-pack),1,3.84,2019-02-13 07:24:00,"18 13th St, San Francisco, CA 94016",13,February,2019,3.84,San Francisco
2,150504,27in 4K Gaming Monitor,1,389.99,2019-02-18 09:46:00,"52 6th St, New York City, NY 10001",18,February,2019,389.99,New York


#### The best month in the year

In [71]:
grouped_data = full_data.groupby('Order Month')['Total Value'].sum()
grouped_data = grouped_data.sort_values(ascending=False)
the_best_month = grouped_data.index[0]
value = grouped_data.values[0]
value = "{:,}".format(value)

print(f"The best month in 2019 was {the_best_month} and we earned {value}")

The best month in 2019 was December and we earned 4,557,905.42


#### The most product sold in the city